In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import math

In [5]:
schedule = pd.read_csv('collected_schedules/best_schedule_bus400_gen1.csv')
schedule.head()

,bus,trip_id
0,0,11452
1,0,10197
2,0,8692
3,0,16481
4,0,10785


In [6]:
trips = pd.read_csv('formatted_DMRC_trips.csv')
trips.head()

,Unnamed: 0,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,start_time_y,end_time_y,start_stop_y,end_stop_y
0,0,0,weekday,0,NaN,NaN,NaN,NaN,shp_1_30,0,0,05:28:08,06:18:38,21,1
1,1,0,weekday,1,NaN,NaN,NaN,NaN,shp_1_30,0,0,05:35:18,06:25:48,21,1
2,2,0,weekday,10,NaN,NaN,NaN,NaN,shp_1_30,0,0,06:39:48,07:30:18,21,1
3,3,0,weekday,100,NaN,NaN,NaN,NaN,shp_1_30,0,0,16:48:04,17:38:34,21,1
4,4,2,weekday,1000,NaN,NaN,NaN,NaN,shp_1_13,0,0,18:26:21,20:00:41,71,36


In [7]:
schedule_trips_merge = pd.merge(schedule, trips, on='trip_id').drop(columns=['service_id','trip_headsign','trip_short_name','direction_id','block_id','wheelchair_accessible','bikes_allowed','Unnamed: 0'])
schedule_trips_merge

,bus,trip_id,route_id,shape_id,start_time_y,end_time_y,start_stop_y,end_stop_y
0,0,11452,22,shp_1_22,06:21:24,06:46:23,62,71
1,0,10197,20,shp_1_31,06:53:04,08:27:24,36,71
2,0,8692,17,shp_1_21,08:27:30,08:55:50,508,520
3,0,16481,33,shp_1_24,08:59:10,09:37:10,153,172
4,0,10785,21,shp_1_34,09:39:59,10:29:06,44,62
...,...,...,...,...,...,...,...,...
5434,399,6132,12,shp_1_16,16:07:35,17:15:47,108,81
5435,399,10315,20,shp_1_31,17:26:24,19:00:44,36,71
5436,399,4201,7,shp_1_17,19:01:00,19:53:33,100,202
5437,399,561,1,shp_1_32,19:59:06,21:08:41,21,225


In [8]:
stops = pd.read_csv('DMRC_Data/stops.txt')
stops_cleaned = stops.drop(columns=['stop_code', 'stop_desc'])
stops_cleaned.head()

,stop_id,stop_name,stop_lat,stop_lon
0,1,Dilshad Garden,28.675991,77.321495
1,2,Jhilmil,28.675648,77.312393
2,3,Mansrover park,28.675352,77.301178
3,4,Shahdara,28.673531,77.287270
4,5,Welcome,28.671986,77.277931


In [9]:
times = pd.read_csv('DMRC_Data/stop_times.txt')
times_cleaned = times.drop(columns=['stop_headsign','pickup_type','drop_off_type','timepoint', 'continuous_pickup', 'continuous_drop_off'])
times_cleaned.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled
0,0,05:28:08,05:28:28,21,0,0.000
1,0,05:30:58,05:31:18,20,1,1202.405
2,0,05:33:28,05:33:48,19,2,2480.750
3,0,05:35:33,05:35:53,18,3,3314.936
4,0,05:37:53,05:38:13,17,4,4300.216


In [10]:
stops_times_merge = pd.merge(times_cleaned, stops_cleaned)
stops_times_merge.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,stop_name,stop_lat,stop_lon
0,0,05:28:08,05:28:28,21,0,0.000,Rithala,28.720821,77.105042
1,0,05:30:58,05:31:18,20,1,1202.405,Rohini West,28.715008,77.115746
2,0,05:33:28,05:33:48,19,2,2480.750,Rohini East,28.707941,77.125732
3,0,05:35:33,05:35:53,18,3,3314.936,Pitampura,28.703180,77.132355
4,0,05:37:53,05:38:13,17,4,4300.216,Kohat Enclave,28.697943,77.140465


In [11]:
trip_distances = stops_times_merge.groupby('trip_id')['shape_dist_traveled'].last() / 1000
schedule_combined = pd.merge(schedule_trips_merge, trip_distances, on='trip_id')
schedule_combined

,bus,trip_id,route_id,shape_id,start_time_y,end_time_y,start_stop_y,end_stop_y,shape_dist_traveled
0,0,11452,22,shp_1_22,06:21:24,06:46:23,62,71,14.527313
1,0,10197,20,shp_1_31,06:53:04,08:27:24,36,71,55.877938
2,0,8692,17,shp_1_21,08:27:30,08:55:50,508,520,17.323959
3,0,16481,33,shp_1_24,08:59:10,09:37:10,153,172,10.425216
4,0,10785,21,shp_1_34,09:39:59,10:29:06,44,62,22.246105
...,...,...,...,...,...,...,...,...,...
5434,399,6132,12,shp_1_16,16:07:35,17:15:47,108,81,35.944730
5435,399,10315,20,shp_1_31,17:26:24,19:00:44,36,71,55.877938
5436,399,4201,7,shp_1_17,19:01:00,19:53:33,100,202,27.284018
5437,399,561,1,shp_1_32,19:59:06,21:08:41,21,225,33.697449


In [12]:
bus_statistics = pd.DataFrame(schedule_combined.groupby('bus')['shape_dist_traveled'].agg('sum'))
bus_statistics.head()

,shape_dist_traveled
bus,
0,452.060221
1,413.856212
2,362.585996
3,478.420887
4,465.808659


In [13]:
def adjust_time_to_next_day(time_str):
    try:
        # Split the time into components
        time_parts = time_str.split(':')
        hour = int(time_parts[0])
        
        # Check if hour is >= 24
        if hour >= 24:
            # Subtract 24 from hours and shift the time to the next day
            adjusted_hour = hour - 24
            adjusted_time_str = f"{adjusted_hour:02}:{time_parts[1]}:{time_parts[2]}"
            return pd.to_datetime(adjusted_time_str) + pd.Timedelta(days=1)
        else:
            return pd.to_datetime(time_str)
    except Exception as e:
        print(f"Error adjusting time: {time_str}, {e}")
        return None

In [14]:
bus_statistics['trip_count'] = schedule_combined.groupby('bus').agg('size')
schedule_combined_sorted = schedule_combined.sort_values(by=['bus', 'start_time_y'])
bus_statistics['travel_time'] = (pd.to_datetime(schedule_combined_sorted.groupby('bus')['end_time_y'].last().apply(adjust_time_to_next_day)) - pd.to_datetime(schedule_combined_sorted.groupby('bus')['start_time_y'].first().apply(adjust_time_to_next_day))).dt.total_seconds() / 3600
bus_statistics.head()

,shape_dist_traveled,trip_count,travel_time
bus,,,
0,452.060221,15,16.048333
1,413.856212,10,17.724722
2,362.585996,14,15.205556
3,478.420887,17,18.604722
4,465.808659,13,16.778333


In [15]:
mileage = 3.7
petrol_price = 94.77
bus_statistics['petrol_charges'] = bus_statistics['shape_dist_traveled'] / mileage * petrol_price

In [16]:
co2_emission_factor = 2.31  
fuel_consumption_per_km = mileage
bus_statistics['carbon_emissions'] = (
    bus_statistics['shape_dist_traveled'] * fuel_consumption_per_km * co2_emission_factor
)

In [17]:
bus_statistics.describe()

,shape_dist_traveled,trip_count,travel_time,petrol_charges,carbon_emissions
count,400.000000,400.000000,400.000000,400.000000,400.000000
mean,410.612792,13.597500,16.850562,10517.236286,3509.507530
std,51.748065,2.488166,1.047582,1325.449772,442.290715
min,207.051466,5.000000,13.377778,5303.315522,1769.668880
25%,376.047102,12.000000,16.288819,9631.887535,3214.074583
50%,413.694714,14.000000,16.926111,10596.175160,3535.848725
75%,440.803516,15.000000,17.547569,11290.526807,3767.547649
max,592.540145,23.000000,19.126944,15177.035011,5064.440619


In [18]:
bus_statistics.head()

,shape_dist_traveled,trip_count,travel_time,petrol_charges,carbon_emissions
bus,,,,,
0,452.060221,15,16.048333,11578.850580,3863.758709
1,413.856212,10,17.724722,10600.311679,3537.229044
2,362.585996,14,15.205556,9287.101308,3099.022508
3,478.420887,17,18.604722,12254.039854,4089.063321
4,465.808659,13,16.778333,11930.996382,3981.266608


In [19]:
stops.drop(columns=['stop_code','stop_desc']).head()

,stop_id,stop_name,stop_lat,stop_lon
0,1,Dilshad Garden,28.675991,77.321495
1,2,Jhilmil,28.675648,77.312393
2,3,Mansrover park,28.675352,77.301178
3,4,Shahdara,28.673531,77.287270
4,5,Welcome,28.671986,77.277931


In [20]:
schedule_combined = schedule_combined.merge(stops.drop(columns=['stop_code', 'stop_desc']),
                                            left_on='start_stop_y',
                                            right_on='stop_id')

In [21]:
schedule_combined = schedule_combined.rename(
    columns={
        'stop_name': 'start_stop_name',
        'stop_lat': 'start_stop_lat',
        'stop_lon': 'start_stop_lon'
    }
).drop(columns='stop_id')
schedule_combined.head()

,bus,trip_id,route_id,shape_id,start_time_y,end_time_y,start_stop_y,end_stop_y,shape_dist_traveled,start_stop_name,start_stop_lat,start_stop_lon
0,0,11452,22,shp_1_22,06:21:24,06:46:23,62,71,14.527313,Qutab Minar,28.512714,77.185791
1,0,10197,20,shp_1_31,06:53:04,08:27:24,36,71,55.877938,Samaypur Badli,28.742872,77.146545
2,0,8692,17,shp_1_21,08:27:30,08:55:50,508,520,17.323959,Noida Sector 142,28.498999,77.412567
3,0,16481,33,shp_1_24,08:59:10,09:37:10,153,172,10.425216,Phase 3 (Rapid Metro),28.494329,77.093552
4,0,10785,21,shp_1_34,09:39:59,10:29:06,44,62,22.246105,Vishwavidyalaya,28.694765,77.212418


In [22]:
schedule_combined = schedule_combined.merge(stops.drop(columns=['stop_code','stop_desc']),left_on='end_stop_y',right_on='stop_id')

In [23]:
schedule_combined = schedule_combined.rename(
    columns={
        'stop_name': 'end_stop_name',
        'stop_lat': 'end_stop_lat',
        'stop_lon': 'end_stop_lon',
        'shape_dist_traveled': 'total_distance',
        'start_time_y' :'start_time',
        'end_time_y' :'end_time',
        'start_stop_y' :'start_stop',
        'end_stop_y' :'end_stop',
    }
).drop(columns='stop_id')
schedule_combined.head()

,bus,trip_id,route_id,shape_id,start_time,end_time,start_stop,end_stop,total_distance,start_stop_name,start_stop_lat,start_stop_lon,end_stop_name,end_stop_lat,end_stop_lon
0,0,11452,22,shp_1_22,06:21:24,06:46:23,62,71,14.527313,Qutab Minar,28.512714,77.185791,Huda City Centre,28.459118,77.072586
1,0,10197,20,shp_1_31,06:53:04,08:27:24,36,71,55.877938,Samaypur Badli,28.742872,77.146545,Huda City Centre,28.459118,77.072586
2,0,8692,17,shp_1_21,08:27:30,08:55:50,508,520,17.323959,Noida Sector 142,28.498999,77.412567,Depot Station,28.488651,77.544075
3,0,16481,33,shp_1_24,08:59:10,09:37:10,153,172,10.425216,Phase 3 (Rapid Metro),28.494329,77.093552,Sector 55-56 (Rapid Metro),28.424587,77.105042
4,0,10785,21,shp_1_34,09:39:59,10:29:06,44,62,22.246105,Vishwavidyalaya,28.694765,77.212418,Qutab Minar,28.512714,77.185791


In [24]:
new_order = ['bus', 'trip_id', 'route_id', 'shape_id', 'start_time', 
       'start_stop', 'start_stop_name',
       'start_stop_lat', 'start_stop_lon', 'end_time','end_stop', 'end_stop_name', 'end_stop_lat',
       'end_stop_lon', 'total_distance']
schedule_combined = schedule_combined[new_order]
schedule_combined.head()

,bus,trip_id,route_id,shape_id,start_time,start_stop,start_stop_name,start_stop_lat,start_stop_lon,end_time,end_stop,end_stop_name,end_stop_lat,end_stop_lon,total_distance
0,0,11452,22,shp_1_22,06:21:24,62,Qutab Minar,28.512714,77.185791,06:46:23,71,Huda City Centre,28.459118,77.072586,14.527313
1,0,10197,20,shp_1_31,06:53:04,36,Samaypur Badli,28.742872,77.146545,08:27:24,71,Huda City Centre,28.459118,77.072586,55.877938
2,0,8692,17,shp_1_21,08:27:30,508,Noida Sector 142,28.498999,77.412567,08:55:50,520,Depot Station,28.488651,77.544075,17.323959
3,0,16481,33,shp_1_24,08:59:10,153,Phase 3 (Rapid Metro),28.494329,77.093552,09:37:10,172,Sector 55-56 (Rapid Metro),28.424587,77.105042,10.425216
4,0,10785,21,shp_1_34,09:39:59,44,Vishwavidyalaya,28.694765,77.212418,10:29:06,62,Qutab Minar,28.512714,77.185791,22.246105


In [25]:
# Fix time format for arrival and departure times
def fix_time_format(time_str):
    """Convert times like 24:xx:xx to 00:xx:xx (next day logic)."""
    try:
        hours, minutes, seconds = map(int, time_str.split(":"))
        if hours >= 24:
            hours -= 24
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    except ValueError:
        return time_str

In [26]:
# Function to calculate time difference considering the next day
def calculate_time_gap(start_time_str, end_time_str):
    # Define the time format
    time_format = "%H:%M:%S"
    
    # Convert time strings to datetime objects
    start_time = datetime.strptime(start_time_str, time_format)
    end_time = datetime.strptime(end_time_str, time_format)
    
    # If end_time is earlier than start_time, it means it's on the next day
    if end_time < start_time:
        end_time += timedelta(days=1)  # Add one day to end_time
    
    # Calculate the time difference
    time_gap = end_time - start_time
    return time_gap

In [27]:
schedule_combined['start_time'] = schedule_combined['start_time'].apply(fix_time_format)
schedule_combined['end_time'] = schedule_combined['end_time'].apply(fix_time_format)

schedule_combined['total_time'] = schedule_combined.apply(
    lambda row: calculate_time_gap(row['start_time'], row['end_time']).total_seconds() / 60, axis=1
)
schedule_combined.head()

,bus,trip_id,route_id,shape_id,start_time,start_stop,start_stop_name,start_stop_lat,start_stop_lon,end_time,end_stop,end_stop_name,end_stop_lat,end_stop_lon,total_distance,total_time
0,0,11452,22,shp_1_22,06:21:24,62,Qutab Minar,28.512714,77.185791,06:46:23,71,Huda City Centre,28.459118,77.072586,14.527313,24.983333
1,0,10197,20,shp_1_31,06:53:04,36,Samaypur Badli,28.742872,77.146545,08:27:24,71,Huda City Centre,28.459118,77.072586,55.877938,94.333333
2,0,8692,17,shp_1_21,08:27:30,508,Noida Sector 142,28.498999,77.412567,08:55:50,520,Depot Station,28.488651,77.544075,17.323959,28.333333
3,0,16481,33,shp_1_24,08:59:10,153,Phase 3 (Rapid Metro),28.494329,77.093552,09:37:10,172,Sector 55-56 (Rapid Metro),28.424587,77.105042,10.425216,38.000000
4,0,10785,21,shp_1_34,09:39:59,44,Vishwavidyalaya,28.694765,77.212418,10:29:06,62,Qutab Minar,28.512714,77.185791,22.246105,49.116667


In [28]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

In [29]:
schedule_combined = schedule_combined.sort_values(by=['bus','start_time'])

# Initialize the column with NaN or None values
schedule_combined['travel_distance_to_next'] = np.zeros(len(schedule_combined))  # Or use np.nan if you prefer

# Loop to calculate 'travel_distance_to_next' for each row
for i in range(0, len(schedule_combined) - 1):
    # Compare trip_id of consecutive rows
    if schedule_combined.iloc[i]['bus'] == schedule_combined.iloc[i+1]['bus']:
        schedule_combined.at[i, 'travel_distance_to_next'] = haversine(
            schedule_combined.iloc[i+1]['end_stop_lat'],
            schedule_combined.iloc[i+1]['end_stop_lon'],
            schedule_combined.iloc[i]['start_stop_lat'],
            schedule_combined.iloc[i]['start_stop_lon']
        )

In [30]:
schedule_combined

,bus,trip_id,route_id,shape_id,start_time,start_stop,start_stop_name,start_stop_lat,start_stop_lon,end_time,end_stop,end_stop_name,end_stop_lat,end_stop_lon,total_distance,total_time,travel_distance_to_next
0,0,11452,22,shp_1_22,06:21:24,62,Qutab Minar,28.512714,77.185791,06:46:23,71,Huda City Centre,28.459118,77.072586,14.527313,24.983333,12.566863
1,0,10197,20,shp_1_31,06:53:04,36,Samaypur Badli,28.742872,77.146545,08:27:24,71,Huda City Centre,28.459118,77.072586,55.877938,94.333333,48.008586
2,0,8692,17,shp_1_21,08:27:30,508,Noida Sector 142,28.498999,77.412567,08:55:50,520,Depot Station,28.488651,77.544075,17.323959,28.333333,31.180109
3,0,16481,33,shp_1_24,08:59:10,153,Phase 3 (Rapid Metro),28.494329,77.093552,09:37:10,172,Sector 55-56 (Rapid Metro),28.424587,77.105042,10.425216,38.000000,9.242225
4,0,10785,21,shp_1_34,09:39:59,44,Vishwavidyalaya,28.694765,77.212418,10:29:06,62,Qutab Minar,28.512714,77.185791,22.246105,49.116667,14.993121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5434,399,6132,12,shp_1_16,16:07:35,108,Janak Puri West,28.629637,77.077866,17:15:47,81,Botanical Garden,28.564198,77.334656,35.944730,68.200000,18.967861
5435,399,10315,20,shp_1_31,17:26:24,36,Samaypur Badli,28.742872,77.146545,19:00:44,71,Huda City Centre,28.459118,77.072586,55.877938,94.333333,22.745076
5436,399,4201,7,shp_1_17,19:01:00,100,Kirti Nagar,28.655773,77.148499,19:53:33,202,Brigadier Hoshiyar Singh,28.697428,76.919128,27.284018,52.550000,26.151833
5437,399,561,1,shp_1_32,19:59:06,21,Rithala,28.720821,77.105042,21:08:41,225,Shaheed Sthal (New Bus Adda),28.670177,77.416031,33.697449,69.583333,13.361200


In [31]:
bus_statistics['active_distance'] = schedule_combined.groupby('bus')['total_distance'].agg('sum')

In [32]:
bus_statistics['active_time'] = schedule_combined.groupby('bus')['total_time'].agg('sum')

In [33]:
bus_statistics['passive_distance'] = schedule_combined.groupby('bus')['travel_distance_to_next'].agg('sum')

In [34]:
bus_statistics['activity_ratio'] = bus_statistics['active_distance'] / bus_statistics['passive_distance']

In [35]:
bus_statistics.describe()

,shape_dist_traveled,trip_count,travel_time,petrol_charges,carbon_emissions,active_distance,active_time,passive_distance,activity_ratio
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,410.612792,13.597500,16.850562,10517.236286,3509.507530,410.612792,835.074958,286.874818,1.508390
std,51.748065,2.488166,1.047582,1325.449772,442.290715,51.748065,98.618964,71.834185,0.372542
min,207.051466,5.000000,13.377778,5303.315522,1769.668880,207.051466,401.400000,105.719361,0.879275
25%,376.047102,12.000000,16.288819,9631.887535,3214.074583,376.047102,775.091667,238.950570,1.254115
50%,413.694714,14.000000,16.926111,10596.175160,3535.848725,413.694714,840.350000,280.444265,1.423106
75%,440.803516,15.000000,17.547569,11290.526807,3767.547649,440.803516,887.316667,336.664066,1.696765
max,592.540145,23.000000,19.126944,15177.035011,5064.440619,592.540145,1159.750000,518.430465,3.459206


In [36]:
bus_statistics.head()

,shape_dist_traveled,trip_count,travel_time,petrol_charges,carbon_emissions,active_distance,active_time,passive_distance,activity_ratio
bus,,,,,,,,,
0,452.060221,15,16.048333,11578.850580,3863.758709,452.060221,875.033333,290.718559,1.554975
1,413.856212,10,17.724722,10600.311679,3537.229044,413.856212,803.033333,221.730978,1.866479
2,362.585996,14,15.205556,9287.101308,3099.022508,362.585996,750.750000,284.359122,1.275099
3,478.420887,17,18.604722,12254.039854,4089.063321,478.420887,947.433333,364.770563,1.311567
4,465.808659,13,16.778333,11930.996382,3981.266608,465.808659,871.900000,239.842950,1.942140
